In [1]:
import sys
sys.path.append('..') # Search for custom module in the top level. 

# Import my custom modules.
from allens_quantum_package.functions import * 
from allens_quantum_package.operators import *

from qiskit import *
from qiskit.quantum_info import state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_experiments.framework import ExperimentData
from qiskit_experiments.library.tomography import MitigatedTomographyAnalysis, MitigatedStateTomography, TomographyAnalysis
from qiskit_experiments.library.tomography.basis import PauliMeasurementBasis
from qiskit_experiments.library.characterization.analysis import LocalReadoutErrorAnalysis
from qiskit_ibm_runtime import QiskitRuntimeService

from numpy import set_printoptions, radians, ndarray
import scipy

In [2]:
# Set the floating point diplay precision to 2 decimal places, sufficient for our purposes.
set_printoptions(precision=2)

# Initialise the Qiskit runtime service. 
service = QiskitRuntimeService()

In [3]:
states = [
    (radians(-88), radians(79)),
    (radians(32), radians(-167)),
    (radians(140), radians(125)),
    
    (radians(-76), radians(-88)),
    (radians(93), radians(-135)),
    (radians(104), radians(-96)),

    (radians(-104), radians(-146)),
    (radians(-158), radians(-108)),
    (radians(-110), radians(-172))
]

In [5]:
def build_circuit(theta, phi) -> QuantumCircuit:
    circ = QuantumCircuit(1)
    circ.u(theta, phi, 0, 0)
    return circ

In [17]:
ibm_brisbane = service.get_backend('ibm_brisbane')

In [55]:
circuits_to_send = list(itertools.chain.from_iterable([
        circuit for circuit in [
            get_tomography_circuits(build_circuit(theta, phi), 0) for theta, phi in states
        ]
    ]
))

In [56]:
circuits_to_send = [transpile(circuit, ibm_brisbane) for circuit in circuits_to_send]

In [57]:
mem_circ_0 = QuantumCircuit(1)
mem_circ_0.measure_all()

mem_circ_1 = QuantumCircuit(1)
mem_circ_1.x(0)
mem_circ_1.measure_all()

circuits_to_send = [mem_circ_0, mem_circ_1] + circuits_to_send

In [58]:
hardware_job = ibm_brisbane.run(circuits=circuits_to_send)

In [64]:
print(f"Hardware job ID: {hardware_job.job_id()}")

Hardware job ID: cvc1sr7z17rg008d0ab0


---

In [4]:
hardware_job = service.job('cvc1sr7z17rg008d0ab0')
result_counts = hardware_job.result().get_counts()

In [5]:
def get_mitigation_matrix(num_qubits: int, zero_counts: dict[str, int], one_counts: dict[str, int]) -> ndarray:

    assignment_matrices = []
    
    for qubit_idx in range(num_qubits):

        # Determine zero state for assignment matrix
        zero_count = sum([count for result, count in zero_counts.items() if result[qubit_idx] == '0'])
        one_count = sum([count for result, count in zero_counts.items() if result[qubit_idx] == '1'])

        # Calculate zero ket
        zero_ket = array([[zero_count], 
                          [one_count]]) / (zero_count + one_count)
        
        # Determine one state for assignment matrix
        zero_count = sum([count for result, count in one_counts.items() if result[qubit_idx] == '0'])
        one_count = sum([count for result, count in one_counts.items() if result[qubit_idx] == '1'])

        # Calculate zero ket
        one_ket = array([[zero_count], 
                         [one_count]]) / (zero_count + one_count)
        
        assignment_matrix = numpy.concatenate([zero_ket, one_ket], axis=1)

        assignment_matrices.append(assignment_matrix)

    return tens(*(scipy.linalg.inv(mat) for mat in assignment_matrices))


def get_bit_strings(count: int) -> list[str]:
    return [''.join(bits) for bits in itertools.product(['0', '1'], repeat=count)]


def mitigate_counts(num_qubits: int, mitigation_matrix: ndarray, counts: dict[str, int]) -> dict[str, int]:
    
    bit_strings = get_bit_strings(num_qubits)

    # Get vector of counts
    counts_vector = numpy.concatenate(
        [array([[counts[bit_string]] if bit_string in counts else 0.0]) for bit_string in bit_strings],
        axis=0
    )

    # Multiply by mitigation matirx
    corrected_vector = mitigation_matrix @ counts_vector
    corrected_vector = corrected_vector.astype(int)

    output = {}
    for idx, bit_string in zip(range(2**num_qubits), bit_strings):
        output[bit_string] = int(corrected_vector[idx])

    return output

In [6]:
def print_fidelities(x_counts, y_counts, z_counts, theta, phi):

    den_op = density_op_from_counts_dict(x_counts, y_counts, z_counts)

    fidelity = state_fidelity(den_op, gen_qubit(theta, phi))

    print(f'State fidelity for \t theta {int(degrees(theta))}°\t phi {int(degrees(phi))}°\t:\t{fidelity}')


In [10]:
print("Unmitigated Fidelities\n----------------------\n")

for count_idx, state_idx in zip(range(4, 30, 3), range(9)):
    print_fidelities(result_counts[count_idx], result_counts[count_idx + 1], result_counts[count_idx + 2], states[state_idx][0], states[state_idx][1])

Unmitigated Fidelities
----------------------

State fidelity for 	 theta -88°	 phi 79°	:	0.9841387312752766
State fidelity for 	 theta 32°	 phi -167°	:	0.9669012303765895
State fidelity for 	 theta 140°	 phi 125°	:	0.9707545810945981
State fidelity for 	 theta -76°	 phi -88°	:	0.9519339301926772
State fidelity for 	 theta 93°	 phi -135°	:	0.9761141503445196
State fidelity for 	 theta 104°	 phi -96°	:	0.9794626674106945
State fidelity for 	 theta -104°	 phi -146°	:	0.9521836633361349
State fidelity for 	 theta -158°	 phi -108°	:	0.975622090156717
State fidelity for 	 theta -110°	 phi -172°	:	0.9603813885263243


In [11]:
def print_mitigated_fidelities(x_counts, y_counts, z_counts, theta, phi):

    x_counts_mitigated = mitigate_counts(1, get_mitigation_matrix(1, result_counts[0], result_counts[1]), x_counts)
    y_counts_mitigated = mitigate_counts(1, get_mitigation_matrix(1, result_counts[0], result_counts[1]), y_counts)
    z_counts_mitigated = mitigate_counts(1, get_mitigation_matrix(1, result_counts[0], result_counts[1]), z_counts)

    den_op = density_op_from_counts_dict(x_counts_mitigated, y_counts_mitigated, z_counts_mitigated)

    fidelity = state_fidelity(den_op, gen_qubit(theta, phi))

    print(f'State fidelity for \t theta {int(degrees(theta))}°\t phi {int(degrees(phi))}°\t:\t{fidelity}')

In [12]:
def bloch_oordinate(counts_dict: dict) -> float:
    plus = counts_dict['0'] if '0' in counts_dict else 0
    minus = counts_dict['1'] if '1' in counts_dict else 0
    return (plus - minus) / (plus + minus)

In [13]:
print("Mitigated Fidelities\n----------------------\n")

for count_idx, state_idx in zip(range(4, 31, 3), range(9)):
    print_mitigated_fidelities(result_counts[count_idx], result_counts[count_idx + 1], result_counts[count_idx + 2], states[state_idx][0], states[state_idx][1])

Mitigated Fidelities
----------------------

State fidelity for 	 theta -88°	 phi 79°	:	0.9991941903399294
State fidelity for 	 theta 32°	 phi -167°	:	0.9999177023150138
State fidelity for 	 theta 140°	 phi 125°	:	0.9933677814360153
State fidelity for 	 theta -76°	 phi -88°	:	0.999967913774716
State fidelity for 	 theta 93°	 phi -135°	:	0.9851925126955755
State fidelity for 	 theta 104°	 phi -96°	:	0.9914074810979137
State fidelity for 	 theta -104°	 phi -146°	:	0.9998028377822136
State fidelity for 	 theta -158°	 phi -108°	:	0.9994925320570449
State fidelity for 	 theta -110°	 phi -172°	:	0.9998962651525923
